Проект сделан на основе выработанного шаблона:
- https://github.com/evgeny-ydsc/first-project/blob/main/шаблон%20проекта/шаблон%20проекта.ipynb

# Название проекта

краткое описание проекта

# Постановка задачи

## Критерии успеха
- ...

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных



- **Целевой признак:**


# Инициализация

## Импорт библиотек

In [46]:
CLASSIFICATION_TASK = 1
REGRESSION_TASK = 0

In [47]:
!pip install phik -q
import phik

In [ ]:
!pip install shap -q
import shap

In [ ]:
if CLASSIFICATION_TASK:
    !pip install lightgbm -q
    import lightgbm as lgb

In [ ]:
if CLASSIFICATION_TASK:
    !pip install category_encoders -q
    from category_encoders import TargetEncoder

In [ ]:
def correct_path (csv_name, folder=""):
    if not os.path.exists(folder + csv_name):
        return '/datasets/' + csv_name
    return folder + csv_name

def def_load_csv(csv, folder="", print_info=True, **kwargs):
    display(f"⬇-------- Таблица {csv}: ----------⬇")
    df = pd.read_csv(correct_path(csv, folder), **kwargs)
    if print_info:
        display(df.info())
        display(df.head())
    return df

def columns_to_snake_case (df):
    names = {}
    for c in df.columns:
        names [c] = re.sub(r'(?<!^)(?=[A-Z])', '_', c).lower() \
            .translate(str.maketrans('','',' (){}[]<>/\\'))
    df.rename(columns=names, inplace=True)
    return df

# визуализирует все категориальные фичи датафрейма в виде гистограммы TOP-значений
# визуализирует все количественные фичи датафрейма в виде ящиков с усами и гистограмм с распределением, выводит descirbe
# поддерживает любое количество фильтров, фильтры поочерёдно применяются к датафрейму как запрос в query()
# если sort_filter_index=-1, то порядок столбцов сортируются их длине (суммарно по фильтрам)
# ignore_columns позволяет избежать рассчёта статистик для столбцов-идентификаторов
def do_eda(df, columns=[], ignore_columns=["id"], filters=[], sort_filter_index=-1, top_categories=10):
    bins=50
    n_filters = len(filters)
    
    if len(columns) == 0:
        columns = df.columns

    # вывод графиков по категориальным фичам 
    for column in columns:
        if ( df[column].dtype == 'O' or df[column].dtype.name == 'category') \
                and column not in ignore_columns:
            display (f"⬇-------- столбец {column} --------⬇")
            title0 = f"столбец {column}, количество и доля значений"
            if n_filters == 0:
                df_show = pd.DataFrame(df.groupby(column)[column].count(),
                    columns = [column])
                df_show.index.name = ''
                df_show = df_show.sort_values( column, ascending=False)
            else:
                df_show = pd.DataFrame()
                for i, _ in enumerate(filters):
                    df_f = df.query(filters[i]).groupby(column)[column].count()
                    df_f = pd.DataFrame({f'фильтр: {filters[i]}':df_f})
                    df_f.index.name = ''
                    df_show = df_show.join(df_f, how='outer')
                df_show = df_show.fillna(0)
                if sort_filter_index == -1:
                    df_show['all_cat_agg_tmp'] = df_show.sum(axis=1)
                df_show = df_show.sort_values( \
                    df_show.columns[sort_filter_index], ascending=False)
                if sort_filter_index == -1:
                    df_show = df_show.drop (columns=['all_cat_agg_tmp'], axis=1)
            # sum not-top categories in a single row
            if top_categories + 1 < df_show.shape[0]:
                df_show_other = df_show.iloc [top_categories:]
                df_show = df_show.iloc [:top_categories]
                df_show.loc[f'All other {df_show_other.shape[0]} categories'] = df_show_other.sum()

            ax = df_show.plot.barh(title=title0, stacked=(n_filters != 0));
            ax.invert_yaxis()
            df_show ['sum_cols'] = df_show.sum(axis=1)
            sum_all = df_show.iloc[:,-1].sum()
            for i in range(df_show.shape[0]):
                sum_row = df_show.iloc[i, -1]
                s = f'{sum_row} ({int(sum_row*100.0/sum_all)}'
                for j in range(n_filters):
                    s += '=' if j == 0 else '+'
                    s += f'{int(df_show.iloc[i,j]*100.0/sum_all)}'
                ax.text(int(sum_row*1.05), i, s+')%')
            xmin, xmax = ax.get_xlim()
            ax.set_xlim(xmin, 1.5*xmax)
            ax.legend(bbox_to_anchor=(1.8, 0.7))
            plt.show();

    # вывод графиков по числовым фичам 
    for column in columns:
        if df[column].dtype in ['int64', 'float64'] and column not in ignore_columns:
            display (f"⬇-------- Следующий признак: {column} --------⬇")
            title = f"столбец {column}"
            _, axs = plt.subplots(nrows=2, ncols=1, figsize=(6, 7))
            if n_filters == 0:
                df_show = df[[column]]
                display(df_show.describe())
            else:
                df_show = pd.DataFrame()
                for i in range (n_filters):
                    df_f = df.query(filters[i]).copy()
                    f_title = f"фильтр: {filters[i]}"
                    df_f.loc[:, 'filter'] = f_title
                    df_f = df_f[['filter', column]]
                    df_show = pd.concat([df_show, df_f], axis=0, ignore_index=True)
                    display(f_title)
                    display(df_f[column].describe())

            ax = sns.boxplot(data=df_show, orient="h", ax=axs[0], y = None if n_filters == 0 else 'filter', x = column)
            sns.histplot(data=df_show, x=column, hue= None if n_filters == 0 else 'filter', multiple="dodge",
                        bins=bins, kde=True, ax=axs[1], alpha=0.8, legend=False)
            plt.show();

def display_drop_full_duplicates(df):
    display(f'{df.duplicated().sum()} full duplicates are dropped')
    return df.drop_duplicates()

def print_unique_text_features(df):
    for c in df.columns:
        if df[c].dtype == 'O':
            x = df[c].unique()
            x = np.sort(x[~pd.isnull(x)])
            display(f"⬇-------- Feature \"{c}\"--------⬇")
            display(x)

# example:
# df = split_date (df, 'created_at', needed_parts='ymdwh', format='%Y-%m-%d %H:%M:%S'),
# use it after solving NaNs to prevent float types of the new fields
def split_date (df, date_column, needed_parts, format="", drop_date_column=False):
    if format != "":
        df[date_column] = pd.to_datetime(df[date_column], format=format)
    if 'y' in needed_parts:
        df [date_column + "_year"] = df[date_column].dt.year
    if 'm' in needed_parts:
        df [date_column + "_month"] = df[date_column].dt.month
    if 'd' in needed_parts:
        df [date_column + "_day"] = df[date_column].dt.day
    if 'w' in needed_parts:
        df [date_column + "_weekday"] = df[date_column].dt.weekday
    if 'h' in needed_parts:
        df [date_column + "_hour"] = df[date_column].dt.hour
    if drop_date_column:
        df = df.drop([date_column], axis=1)
    return df

def show_nans (df):
    n = df.isna().sum()
    l = df.shape[0]
    display("empty parts:")
    display(n [n>0].map(lambda x: f'{round(x/l*100, 2)}%'))

def fill_by_median (df, columns):
    for column in columns:
        df[column] = df[column].fillna(df[column].median())
    return df

def fill_by_value (df, columns, val):
    for column in columns:
        df[column] = df[column].fillna(val)
    return df

def get_multicollinear_features (phik_matrix):
    mc_features=[]
    while True:
        if phik_matrix.shape[0] == 0:
            break
        mc_features_ranked = phik_matrix[(phik_matrix >= 0.9) & (phik_matrix < 1)].sum()
        mc_features_ranked = mc_features_ranked[mc_features_ranked>0] \
            .sort_values(ascending=False)
        if mc_features_ranked.shape[0] == 0:
            break
        top_mc_feature = mc_features_ranked.index[0]
        phik_matrix = phik_matrix.drop(top_mc_feature,axis=0)
        phik_matrix = phik_matrix.drop(top_mc_feature,axis=1)
        mc_features.append(top_mc_feature)
    return mc_features, phik_matrix

## Установка главных параметров

In [51]:
RANDOM_STATE = 42

# Вспомогательные функции

In [ ]:
def correct_path (csv_name, folder=""):
    if not os.path.exists(folder + csv_name):
        return '/datasets/' + csv_name
    return folder + csv_name

def def_load_csv(csv, folder="", print_info=True):
    display(f"⬇-------- Таблица {csv}: ----------⬇")
    df = pd.read_csv(correct_path(csv, folder))
    if print_info:
        display(df.info())
        display(df.head())
    return df

def columns_to_snake_case (df):
    names = {}
    for c in df.columns:
        names [c] = re.sub(r'(?<!^)(?=[A-Z])', '_', c).lower() \
            .translate(str.maketrans('','',' (){}[]<>/\\'))
    df.rename(columns=names, inplace=True)
    return df

# визуализирует все категориальные фичи датафрейма в виде гистограммы TOP-значений
# визуализирует все количественные фичи датафрейма в виде ящиков с усами и гистограмм с распределением, выводит descirbe
# поддерживает любое количество фильтров, фильтры поочерёдно применяются к датафрейму как запрос в query()
# если sort_filter_index=-1, то порядок столбцов сортируются их длине (суммарно по фильтрам)
# ignore_columns позволяет избежать рассчёта статистик для столбцов-идентификаторов
def do_eda(df, columns=[], ignore_columns=["id"], filters=[], sort_filter_index=-1, top_categories=10):
    bins=25
    n_filters = len(filters)
    
    if len(columns) == 0:
        columns = df.columns

    # вывод графиков по категориальным фичам 
    for column in columns:
        if ( df[column].dtype == 'O' or df[column].dtype.name == 'category') \
                and column not in ignore_columns:
            display (f"⬇-------- столбец {column} --------⬇")
            title0 = f"столбец {column}, количество и доля значений"
            if n_filters == 0:
                df_show = pd.DataFrame(df.groupby(column)[column].count(),
                    columns = [column])
                df_show.index.name = ''
                df_show = df_show.sort_values( column, ascending=False)
            else:
                df_show = pd.DataFrame()
                for i, _ in enumerate(filters):
                    df_f = df.query(filters[i]).groupby(column)[column].count()
                    df_f = pd.DataFrame({f'фильтр: {filters[i]}':df_f})
                    df_f.index.name = ''
                    df_show = df_show.join(df_f, how='outer')
                df_show = df_show.fillna(0)
                if sort_filter_index == -1:
                    df_show['all_cat_agg_tmp'] = df_show.sum(axis=1)
                df_show = df_show.sort_values( \
                    df_show.columns[sort_filter_index], ascending=False)
                if sort_filter_index == -1:
                    df_show = df_show.drop (columns=['all_cat_agg_tmp'], axis=1)
            # sum not-top categories in a single row
            if top_categories + 1 < df_show.shape[0]:
                df_show_other = df_show.iloc [top_categories:]
                df_show = df_show.iloc [:top_categories]
                df_show.loc[f'All other {df_show_other.shape[0]} categories'] = df_show_other.sum()

            ax = df_show.plot.barh(title=title0, stacked=(n_filters != 0));
            ax.invert_yaxis()
            df_show ['sum_cols'] = df_show.sum(axis=1)
            sum_all = df_show.iloc[:,-1].sum()
            for i in range(df_show.shape[0]):
                sum_row = df_show.iloc[i, -1]
                s = f'{sum_row} ({int(sum_row*100.0/sum_all)}'
                for j in range(n_filters):
                    s += '=' if j == 0 else '+'
                    s += f'{int(df_show.iloc[i,j]*100.0/sum_all)}'
                ax.text(int(sum_row*1.05), i, s+')%')
            xmin, xmax = ax.get_xlim()
            ax.set_xlim(xmin, 1.5*xmax)
            ax.legend(bbox_to_anchor=(1.8, 0.7))
            plt.show();

    # вывод графиков по числовым фичам 
    for column in columns:
        if df[column].dtype in ['int64', 'float64'] and column not in ignore_columns:
            display (f"⬇-------- Следующий признак: {column} --------⬇")
            title = f"столбец {column}"
            _, axs = plt.subplots(nrows=2, ncols=1, figsize=(6, 7))
            if n_filters == 0:
                df_show = df[[column]]
                display(df_show.describe())
            else:
                df_show = pd.DataFrame()
                for i in range (n_filters):
                    df_f = df.query(filters[i]).copy()
                    f_title = f"фильтр: {filters[i]}"
                    df_f.loc[:, 'filter'] = f_title
                    df_f = df_f[['filter', column]]
                    df_show = pd.concat([df_show, df_f], axis=0, ignore_index=True)
                    display(f_title)
                    display(df_f[column].describe())

            ax = sns.boxplot(data=df_show, orient="h", ax=axs[0], y = None if n_filters == 0 else 'filter', x = column)
            sns.histplot(data=df_show, x=column, hue= None if n_filters == 0 else 'filter', multiple="dodge",
                        bins=bins, kde=True, ax=axs[1], alpha=0.8, legend=False)
            plt.show();

def display_drop_full_duplicates(df):
    display(f'{df.duplicated().sum()} full duplicates are dropped')
    return df.drop_duplicates()

def print_unique_text_features(df):
    for c in df.columns:
        if df[c].dtype == 'O':
            x = df[c].unique()
            x = np.sort(x[~pd.isnull(x)])
            display(f"⬇-------- Feature \"{c}\"--------⬇")
            display(x)

# example:
# df = split_date (df, 'created_at', needed_parts='ymdwh', format='%Y-%m-%d %H:%M:%S'),
# use it after solving NaNs to prevent float types of the new fields
def split_date (df, date_column, needed_parts, format="", drop_date_column=False):
    if format != "":
        df[date_column] = pd.to_datetime(df[date_column], format=format)
    if 'y' in needed_parts:
        df [date_column + "_year"] = df[date_column].dt.year
    if 'm' in needed_parts:
        df [date_column + "_month"] = df[date_column].dt.month
    if 'd' in needed_parts:
        df [date_column + "_day"] = df[date_column].dt.day
    if 'w' in needed_parts:
        df [date_column + "_weekday"] = df[date_column].dt.weekday
    if 'h' in needed_parts:
        df [date_column + "_hour"] = df[date_column].dt.hour
    if drop_date_column:
        df = df.drop([date_column], axis=1)
    return df

def show_nans (df):
    n = df.isna().sum()
    l = df.shape[0]
    display("empty parts:")
    display(n [n>0].map(lambda x: f'{round(x/l*100, 2)}%'))

def fill_by_median (df, columns):
    for column in columns:
        df[column] = df[column].fillna(df[column].median())
    return df

def fill_by_value (df, columns, val):
    for column in columns:
        df[column] = df[column].fillna(val)
    return df

def get_multicollinear_features (phik_matrix):
    mc_features=[]
    while True:
        if phik_matrix.shape[0] == 0:
            break
        mc_features_ranked = phik_matrix[(phik_matrix >= 0.9) & (phik_matrix < 1)].sum()
        mc_features_ranked = mc_features_ranked[mc_features_ranked>0] \
            .sort_values(ascending=False)
        if mc_features_ranked.shape[0] == 0:
            break
        top_mc_feature = mc_features_ranked.index[0]
        phik_matrix = phik_matrix.drop(top_mc_feature,axis=0)
        phik_matrix = phik_matrix.drop(top_mc_feature,axis=1)
        mc_features.append(top_mc_feature)
    return mc_features, phik_matrix

# Подготовка данных

## Загрузка таблиц, исправление сепараторов

In [53]:
#df = def_load_csv ('....csv')

## Поправим имена полей

In [54]:
'''
df = columns_to_snake_case(df)
display (df.info())
display (df.head())
'''

'\ndf = columns_to_snake_case(df)\ndisplay (df.info())\ndisplay (df.head())\n'

## Исправим типы данных

In [55]:
#df['...'] = pd.to_datetime( df['...'], format='%Y-%m-%d %H:%M:%S' )
#df['...'] = df['...'].astype('object')
#df['..._year'] = df['...'].dt.year
#df['..._month'] = df['...'].dt.month

In [56]:
'''
display(df.info())
display(df.head())
'''

'\ndisplay(df.info())\ndisplay(df.head())\n'

## Удалим поля, приводящие к утечке данных или заведомо бесполезные

- `...` - ... комментарий

In [57]:
#df = df.drop(['...', '...'], axis=1)

## Изучим пропуски

In [ ]:
'''
show_nans (df)
'''

'\nn = df.isna().sum()\nl = len(df)\nround(n [n>0] / l, 2)\n'

In [59]:
'''
df['vehicle_type'] = df['vehicle_type'] \
    .fillna(df.groupby(['brand', 'model'])['vehicle_type'] \
            .transform(lambda x: x.mode()[0] if not x.mode().empty else None))
'''

"\ndf['vehicle_type'] = df['vehicle_type']     .fillna(df.groupby(['brand', 'model'])['vehicle_type']             .transform(lambda x: x.mode()[0] if not x.mode().empty else None))\n"

👷🚩[todo] если же какие-то пропуски всё-таки останутся - избавимся от них с помощью imputer-ов в pipeline

## Изучим и удалим явные дубликаты в датафрейме

In [60]:
'''
display(df.duplicated().sum())
df = df.drop_duplicates()
'''

'\ndisplay(df.duplicated().sum())\ndf = df.drop_duplicates()\n'

## Изучим неявные дубликаты в датафрейме


In [61]:
#print_unique_text_features(df)

Заменим значения xxx фичи f на yyy.

In [ ]:
#df.loc[df['f']=='xxx', 'f'] = 'yyy'

## Сравним соответствующие фичи в тренировочной и тестовой выборках если они из разных источников, а не из одной таблицы(состав, типы, значения) 


# Исследовательский анализ данных (EDA)

In [ ]:
#do_eda(df)

## Выводы и задачи по итогам EDA
1. ...
    1. 👷🚩[todo] ...

Скорректируем данные, заменив аномалии:

In [2]:
'''
df.loc[(df['xxx'] <10) | (df['xxx'] > 1600), 'xxx'] = \
    df.groupby(['brand', 'model'])['power'] \
        .transform(lambda x: x.mode()[0] if not x.mode().empty else None)
'''

"\ndf.loc[(df['xxx'] <10) | (df['xxx'] > 1600), 'xxx'] =     df.groupby(['brand', 'model'])['power']         .transform(lambda x: x.mode()[0] if not x.mode().empty else None)\n"

Скорректируем данные, удалив аномалии:

In [ ]:
#df = df.query('...')

Удалим ненужные столбцы:

In [ ]:
'''
columns_to_delete = ['...', '...']
df = df.drop(columns_to_delete, axis=1)
'''

"\ncolumns_to_delete = ['...', '...']\nX_train = X_train.drop(columns_to_delete, axis=1)\nX_test = X_test.drop(columns_to_delete, axis=1)\n"

# Проверим мультиколлинеарность

In [ ]:
'''
interval_cols=['xxx', 'xxx', 'xxx', 'xxx']
   
bins = {}
for c in interval_cols:
    bins[c] = 10;

_, ax = plt.subplots(figsize=(25,25))
ph_mx = X_train.phik_matrix(interval_cols=interval_cols, bins=bins)
sns.heatmap(ph_mx, annot=True, fmt='.2f', ax=ax);

columns_to_delete, ph_mx_no_mc = get_multicollinear_features (ph_mx)
display ("Мультиколлинеарные фичи для удаления:")
display (columns_to_delete)

'''

"\n_, ax = plt.subplots(figsize=(25,25))\nph_mx = X_train.phik_matrix(interval_cols=['kilometer', 'power', 'registration_year'],\n                bins={'kilometer':10, 'power':10, 'registration_year':10})\nsns.heatmap(ph_mx, annot=True, fmt='.2f', ax=ax);\n"

*Как правило, о мультиколлинеарности говорят при значении коэффициента корреляции от 0.9 до 0.95 по модулю. В такой ситуации лишние коррелирующие признаки нужно удалять из обучающей выборки.*

Удалим ненужные столбцы:

In [74]:
'''
columns_to_delete = ['...', '...']
X_train = X_train.drop(columns_to_delete, axis=1)
X_test = X_test.drop(columns_to_delete, axis=1)
'''

"\ncolumns_to_delete = ['...', '...']\nX_train = X_train.drop(columns_to_delete, axis=1)\nX_test = X_test.drop(columns_to_delete, axis=1)\n"

# Разобъём выборку на тренировочную и тестовую (со стратификацией по целевому признаку в случае классификации)

In [ ]:
'''
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    random_state=RANDOM_STATE
)
'''

"\nX = df.drop(['price'], axis=1)\ny = df['price']\n\nX_train, X_test, y_train, y_test = train_test_split(\n    X, \n    y, \n    random_state=RANDOM_STATE\n)\n"

# Проверим сбалансированность тренировочной выборки относительно целевого класса (в случае классификации)

In [ ]:
#y_train.value_counts(normalize=True)

# Обучение моделей

## Pipeline

In [76]:
#cat_columns = ['...', '...']

In [77]:
'''
for c in X_train.columns:
    if X_train[c].dtype == 'O':
        X_train[c] = X_train[c].astype("category")
display(X_train.info())
'''

'\nfor c in X_train.columns:\n    if X_train[c].dtype == \'O\':\n        X_train[c] = X_train[c].astype("category")\ndisplay(X_train.info())\n'

In [ ]:
'''cat_columns = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
ord_columns = ['weather_1', 'road_surface', 'road_condition_1', 'lighting ', 'control_device']
cat_columns = list(set(cat_columns) - set(ord_columns))
num_features = X_train.select_dtypes(include=["number"]).columns.tolist()
'''

In [ ]:
'''
ohe_pipe = Pipeline(
    [('simpleImputer_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ])

ord_pipe = Pipeline(
    [('simpleImputer_before_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ord',  OrdinalEncoder(
                categories=[
                    ['clear', 'cloudy', 'other', '-', 'wind', 'raining', 'snowing', 'fog'], 
                    ['dry', 'wet', '-', 'slippery', 'snowy'], 
                    ['normal', 'other', '-', 'reduced width', 'flooded', 'holes', 'loose material', 'obstruction', 'construction'], 
                    ['daylight', 'dark with street lights', '-', 'dark with street lights not functioning', 'dark with no street lights', 'dusk or dawn'], 
                    ['functioning', 'obscured', 'not functioning', 'none']
                ], 
                handle_unknown='use_encoded_value', unknown_value=np.nan
            )
        ),
     ('simpleImputer_after_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
    ]
)

data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, cat_columns),
        ('ord', ord_pipe, ord_columns),
        ('num', StandardScaler(), num_columns),
    ], 
    remainder='passthrough'
) 
'''

"\ncat_pipe = Pipeline(\n    [('simpleImputer_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),\n     ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))\n    ]\n    )\n\ndata_preprocessor = ColumnTransformer(\n    [\n        ('ohe', cat_pipe, cat_columns),\n        ('num', StandardScaler(), num_columns),\n    ], \n    remainder='passthrough'\n) \n"

In [ ]:
'''
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
]
)
'''


В случае классификации:

In [ ]:
'''
%%time
param_grid = [
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
        'models__max_depth': [4,7],
        'models__max_features': [10, 50],
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
    }
    ,
    {
        'models': [RandomForestClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': [10, 20],
        'models__min_samples_split': [5, 10],
        'models__min_samples_leaf': [2, 4],
        'models__bootstrap': [True, False],
        'preprocessor__num': [MinMaxScaler()]   
    }
    ,
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear', 
            penalty='l1',
            class_weight='balanced'
        )],
        'models__C': range(1, 3),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
    }
    ,
    {
        'models': [SVC(
            random_state=RANDOM_STATE, 
            kernel='linear', probability=True, class_weight='balanced'
        )],
        'models__kernel': ['poly', 'rbf'],
        'models__degree': [2, 5, 10],
        'models__C': range(2, 3),
    }
]

search = RandomizedSearchCV(
    pipe_final, 
    param_grid,
    cv=3,
    scoring='f1',
    n_iter=30,
    verbose = 10,
    n_jobs=-1
)

search.fit(X_train, y_train)

grid_search.best_params_

best_model = search.best_estimator_
'''

"\n%%time\n\npipe_final = Pipeline([\n    ('preprocessor', data_preprocessor),\n    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))\n]\n)\n\nparam_grid = [\n    {\n        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced')],\n        'models__max_depth': [4,7],\n        'models__max_features': [10, 50],\n        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   \n    }\n    ,\n    {\n        'models': [RandomForestClassifier(random_state=RANDOM_STATE)],\n        'models__max_depth': [10, 20],\n        'models__min_samples_split': [5, 10],\n        'models__min_samples_leaf': [2, 4],\n        'models__bootstrap': [True, False],\n        'preprocessor__num': [MinMaxScaler()]   \n    }\n    ,\n    {\n        'models': [LogisticRegression(\n            random_state=RANDOM_STATE, \n            solver='liblinear', \n            penalty='l1',\n            class_weight='balanced'\n        )],\n        'models__C': range(1, 3),\n        '

В случае регрессии:

In [ ]:
'''
param_grid = [
    {
        'models': [LinearRegression()],
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
    }
    ,
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'models__max_depth': [3, 5, 10, None],
        'models__min_samples_split': [2, 5, 10],
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
    },
    {
        'models': [SVR()],
        'models__kernel': ['linear', 'rbf'],
        'models__C': [0.1, 1, 10],
        'models__gamma': ['scale', 'auto']
    },
    {
        'models': [RandomForestRegressor(random_state=RANDOM_STATE)],
        'models__n_estimators': [50, 100],
        'models__max_depth': [None, 10, 20],
        'models__min_samples_split': [2, 5]
    }
]
'''

# Анализ результатов

## Метрики качества на валидационной выборке

In [81]:
#display ('Метрика качества лучшей модели на кроссвалидации: ', search.best_score_)

## Метрики качества на тестовой выборке

In [82]:
# оценка классификации
'''
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
'''

'\ny_pred = best_model.predict(X_test)\nprint(classification_report(y_test, y_pred))\n'

In [83]:
# оценка регрессии
'''
y_pred = best_model.predict(X_test)
display ('RMSE на тестовой выборке = ', root_mean_squared_error(y_test, y_pred))
'''

"\ny_pred = best_model.predict(X_test)\ndisplay ('RMSE на тестовой выборке = ', root_mean_squared_error(y_test, y_pred))\n"

In [ ]:
# ROC Curve
'''
RocCurveDisplay.from_estimator(best_model, X_test, y_test);
'''

In [ ]:
# PrecisionRecallDisplay
'''
RocCurveDisplay.from_estimator(best_model, X_test, y_test);
'''

## Анализ остатков (в случае регрессии)

In [ ]:
'''
residuals = y_test - y_pred

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
axes[0].hist(residuals, bins=33)
axes[0].set_title('Гистограмма распределения остатков')
axes[0].set_xlabel('Остатки')

axes[1].scatter(y_pred, residuals)
axes[1].set_xlabel('Предсказания модели')
axes[1].set_ylabel('Остатки')
axes[1].set_title('Анализ дисперсии')
plt.show()
'''

"\nresiduals = y_test - y_pred\n\nfig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))\naxes[0].hist(residuals, bins=33)\naxes[0].set_title('Гистограмма распределения остатков')\naxes[0].set_xlabel('Остатки')\n\naxes[1].scatter(predictions, residuals)\naxes[1].set_xlabel('Предсказания модели')\naxes[1].set_ylabel('Остатки')\naxes[1].set_title('Анализ дисперсии')\nplt.show()\n"

Остатки модели должны быть нормально распределены, симметрично располагаться на графике, а также иметь постоянную дисперсию для всех значений предсказаний.

Для всех случаев ненормального распределения данных подходят два совета:
- Найти и добавить в модель дополнительные признаки. Скорее всего, на этапе сбора данных не была учтена какая-то информация.
- Трансформировать данные, чтобы изменить взаимосвязь входных признаков с целевым. Этим преобразованиям вы научитесь в следующем

## Анализ важности фичей

In [85]:
'''
samples = 20
 
# Извлечение лучшей модели из результатов RandomizedSearchCV
best_model = search.best_estimator_.named_steps['models']

# Предобработка данных через пайплайн без конечной модели
preprocessor = search.best_estimator_.named_steps['preprocessor']
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

all_feature_names = preprocessor.get_feature_names_out()

# Создаем DataFrame с соответствующими именами колонок
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, 
                                       columns=all_feature_names)
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, 
                                      columns=all_feature_names)

# Семпл данных для KernelExplainer
X_train_preprocessed_smpl = shap.sample(X_train_preprocessed_df, 
                                        samples, random_state=RANDOM_STATE)
X_test_preprocessed_smpl = shap.sample(X_test_preprocessed_df, 
                                       samples, random_state=RANDOM_STATE)

# Теперь, когда у нас есть DataFrame с именами признаков, мы можем использовать KernelExplainer
###explainer = shap.KernelExplainer(best_model.predict_proba, X_train_preprocessed_smpl)
explainer = shap.KernelExplainer(best_model.predict_proba, X_train_preprocessed_smpl)
shap_values = explainer.shap_values(X_test_preprocessed_smpl)

# Создаем объект Explanation для первого КЛАССА.
shap_values_explanation = shap.Explanation(
    values=shap_values[:,:,1], 
    base_values=explainer.expected_value,
    data=X_test_preprocessed_smpl,
    feature_names=all_feature_names
)

shap.summary_plot(shap_values_explanation, plot_size=[15,6])
plt.show()
'''

"\nsamples = 20\n \n# Извлечение лучшей модели из результатов RandomizedSearchCV\nbest_model = search.best_estimator_.named_steps['models']\n\n# Предобработка данных через пайплайн без конечной модели\npreprocessor = search.best_estimator_.named_steps['preprocessor']\nX_train_preprocessed = preprocessor.transform(X_train)\nX_test_preprocessed = preprocessor.transform(X_test)\n\nall_feature_names = preprocessor.get_feature_names_out()\n\n# Создаем DataFrame с соответствующими именами колонок\nX_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, \n                                       columns=all_feature_names)\nX_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, \n                                      columns=all_feature_names)\n\n# Семпл данных для KernelExplainer\nX_train_preprocessed_smpl = shap.sample(X_train_preprocessed_df, \n                                        samples, random_state=RANDOM_STATE)\nX_test_preprocessed_smpl = shap.sample(X_test_preprocessed_df, \

In [86]:
'''
shap.summary_plot(shap_values_explanation, plot_size=[15,6], plot_type='bar')
plt.show()
'''

"\nshap.summary_plot(shap_values_explanation, plot_size=[15,6], plot_type='bar')\nplt.show()\n"

In [87]:
'''
shap.plots.waterfall(shap_values_explanation[0]);
shap.plots.waterfall(shap_values_explanation[1]);
plt.show()
'''

'\nshap.plots.waterfall(shap_values_explanation[0]);\nshap.plots.waterfall(shap_values_explanation[1]);\nplt.show()\n'

# Отчёт и выводы


- В соответствии с поставленной задачей(см. первый раздел)
    - была проведена подготовка данных и их разведочный анализ, в результате которых:
        - были исправлены имена полей, исправлены типы данных, удалены малополезные признаки (в т. ч. с учётом мультиколлинеарности), заполнены пропуски, удалены явные и неявные дубликаты, ...
    - была натренирована модель, предсказывающая ...
        - выбрана наиболее оптимальная модель - **...**, показавшая наилучшие результаты:
            - качество предсказания: ...
            - это удовлетворяет заданный критерий качества: ...

# Appendix - comments legend

These icons and [tags] were used to simplify visualization and searching for conclusions, open questions and subtasks, not to forget to finish/check something important. It's convenient to copy a cell from here and paste it into the right place, adding details and then find it by tags or visually.

👷🚩[todo] 

⬆ this task is to be done

👷✅ 

⬆ the task has been done

👷🚨[SOS] 

⬆ need help

👷🔔[reminder] 

⬆ some work to do or check later

⚠️[!] 

⬆ important constraints or facts

💡 [!] 

⬆ something interesting

🏭 [data transformation] this change ⤴ should be done before training/testing/inference

👷🚧🚧🚧🚧🚧 [in progress] 

⬆ in progress right now

🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧

➡️ some code to be reworked ⬅️

🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧